<a href="https://colab.research.google.com/github/kithmini-wijesiri/De-Novo-Molecular-Generation-using-gflownet/blob/main/Graph_GA_GFlowNet_and_safe_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GraphGA**

A graph-based genetic algorithm (GraphGA) is a variation of a genetic algorithm (GA) that operates on molecular graphs rather than traditional linear representations (like SMILES). It combines the principles of genetic algorithms—such as evolution through selection, mutation, and crossover—with the graph-based representation of molecules, where atoms are nodes and bonds are edges in a graph.

Key Concepts of GraphGA:

*1. Graph Representation of Molecules*

In GraphGA, molecules are represented as graphs where:
Nodes represent atoms.
Edges represent bonds between atoms.
This is a more natural representation of chemical structures compared to string-based formats like SMILES, allowing for more sophisticated manipulations during mutation and crossover operations.

*2. Population*

Like in a traditional genetic algorithm, GraphGA starts with an initial population of molecules (graphs). This population evolves over generations to improve certain properties, such as drug-likeness, binding affinity, or other objectives.
Each molecule in the population is a graph structure.

*3. Fitness Function*

The fitness function evaluates each molecule in the population based on desired properties (e.g., binding affinity, toxicity, or synthetic accessibility).
In the case of molecular design, fitness functions might evaluate predicted biological activity (such as binding to a target protein like EGFR) or other chemical properties (e.g., molecular weight, logP, etc.).
The fitness score is used to guide selection in the next steps.

*4. Selection*

Molecules with higher fitness scores are selected for reproduction to generate new molecules. Selection can be done through methods like roulette wheel or tournament selection, favoring fitter molecules for the next generation.

*5. Crossover (Recombination)*

Crossover is the process by which two parent molecules (graphs) are combined to create new offspring molecules.
For molecules, this could mean combining subgraphs from two parent molecules (e.g., taking a fragment from one molecule and replacing part of another molecule with that fragment).
The key challenge here is maintaining valid molecular structures, ensuring that the offspring graph represents a chemically valid molecule.

*6. Mutation*

Mutation introduces diversity into the population by randomly modifying the molecular graph.
In a molecular graph, mutation could involve:
Adding or removing bonds between atoms.
Replacing atoms with other types.
Adding new substructures (functional groups or rings).
The mutation must ensure that the resulting structure remains chemically valid.

*7. Termination*

The algorithm continues for a certain number of generations or until some other stopping criterion is met (e.g., finding a molecule that meets the desired properties or observing convergence in the population's fitness scores).

*8. Offspring Population*

After crossover and mutation, a new offspring population is created. These new molecules are evaluated by the fitness function, and the process repeats for multiple generations, evolving towards better solutions.

# **GFlowNet**

GFlowNets, or generative flow networks, are a class of energy-based models that work well on discrete combinatorial problems, like generating molecules. In other words, instead of randomly editing molecules and filtering them by score, we will train a model that gets rewarded for producing molecules with a high score. GFlowNets have built-in mechanisms for diversity, making them a more efficient way to more explore the chemical space.

The main idea behind GFlowNet is to interpret the DAG (Directed Acyclic Graph) of possible constructions of an object as a **flow network**, and to think of each edge in that DAG as a pipe through which some amount of water, or particles, _flows_. We then want to _find_ a flow where, (a) flow is preserved, (b) the flow coming into a terminal state (a finished object) is equal to its reward (every non-terminal state has a reward of 0).

### The GFlowNet Objectives

A central finding of the [first GFlowNet paper](https://arxiv.org/abs/2106.04399) is that if we assign to each edge a _flow_ $F(s,s')>0$, and that this flow respects the following property:
$$\forall s', \sum_{s: (s,s')\in G} F(s,s') = \sum_{s'':(s',s'')\in G} F(s',s'')$$
which ensures _flow consistency_ (i.e. the inward flow to a node should match the outward flow from that node; the outward flow from a terminal node is the reward $R(x)$ ) and define the following policy:
$$P_F(s'|s) = \frac{F(s, s')}{\sum_{s''}F(s, s'')}$$
and sample trajectories (paths through the DAG) using this policy, we will sample terminal states (finished objects $x$) with probability:
$$p(x) \propto R(x)$$

Another point of view we can adopt is to think of the flow of a _trajectory_ $\tau$. If you think of all the possible paths from $s_0$ the initial state to some state $s$, they can all be "assigned" some amount of flow (just like we "assigned" flows to edges earlier). Just like we wanted all incoming flows to a terminal state to be equal to its reward $R(x)$, we can say we want all the trajectory flows that go to a state to equal its reward. More generally we can write
$$F(s) = \sum_{\tau, s\in\tau} F(\tau)$$

Let's also write the probability of a particular trajectory as the product of its policy steps:
$$P(\tau) = \prod_{(s,s') \in \tau} P_F(s'|s) =\frac{1}{Z}F(\tau)$$
we note the relationship of $P(\tau)$ with $F(\tau)$: "what fraction of the total flow $Z$ goes through $\tau$?".

Yet another equivalent point of view we can adopt is to think of _backward_ trajectories. We can define $P_B$ as a distribution over the parents of a state, with the following property

$$P(\tau) = \prod_{(s,s') \in \tau} P_B(s|s')$$

By equating the probability of a trajectory $\tau=(s_0,...,s_n=x)$ using the backward policy to that same probability using the forward policy, we end up with this equality:


$$Z \prod_{t} P_F(s_{t+1}|s_t) = R(x)\prod_t P_B(s_t|s_{t+1})$$

Taking the log and squaring the difference of both sides yields the **Trajectory Balance** objective, which we'll use in this lab.

For more general introductions to GFlowNets, check out the following:
- The 2023 [GFlowNet workshop](https://gflownet.org) has several introductory talks and colab tutorials.
- This high-level [GFlowNet colab tutorial](https://colab.research.google.com/drive/1fUMwgu2OhYpQagpzU5mhe9_Esib3Q2VR) (updated versions of which were written for the 2023 workshop, in particular for continuous GFNs).

For this project we'll use Recursion's [`gflownet`](https://github.com/recursionpharma/gflownet) library, which implements all both the definition of the DAG and the training objectives.

What is **Fragment-Based Molecular Design?**

Fragment-based molecular design (FBMD) involves building molecules by combining small, chemically diverse fragments (substructures). These fragments serve as the building blocks for larger, more complex molecules. The idea is to iteratively assemble fragments into molecules that possess desirable properties, such as binding strongly to a biological target like a protein.

FBMD is useful in drug discovery because:

It allows for the design of novel molecules that may not be in existing chemical databases.
Fragments are often easier to optimize since their properties can be precisely tuned before combining them into larger molecules.

# **safe-gpt**

**SAFE-GPT** stands for Safety-Enhanced Generative Pre-trained Transformer. It is an adaptation of the popular GPT (Generative Pre-trained Transformer) model, often used in natural language processing (NLP), to generate molecules. The idea is to apply the capabilities of language models to generate molecules while incorporating safety mechanisms to ensure that the generated molecules are chemically valid, non-toxic, and meet other criteria necessary for drug discovery.

**Some interesting config flags**

There's a plethora of flags in the library, and while they are documented (in `config.py` files), here are some relevant ones.

Algorithm flags
- `algo.illegal_action_logreward`, this acts as both the log-reward for "illegal actions" (which are impossible in many environments) and the minimum possible log-reward. The default of `-100` is easily attained if `beta` is high enough, e.g. for `beta=64`, this corresponds to a minimum reward of $0.21$.
- `algo.train_random_action_prob`, a commonly used setting in GFlowNet is to take some actions at random which a small chance (e.g. $0.1$ or $0.01$) during training to encourage exploration.
- `algo.sampling_tau`, another [common trick](https://arxiv.org/abs/2310.19685) is to maintain a delayed set of parameters which are used to sample new trajectories, updated as $\theta' \leftarrow \tau\theta' + (1-\tau)\theta$ after every gradient step. Typical useful values are in $[0.9,0.99]$

GFN/TB flags
- `algo.tb.do_parameterize_p_b`, the default $P_B$ is the uniform policy. Turn this flag on to instead learn it.
- `algo.tb.variant`, the library implements `TB`, `SubTB1`, and `DB` (flow matching is a separate class)
- `algo.tb.do_correct_idempotent`, (warning, expensive!) activating this flag makes sure that [different actions that lead to the same state](https://openreview.net/forum?id=CZGHAeeBk3) are counted together appropriately.

Replay flags:
- `replay.use`, flag to turn replay on or off
- `replay.capacity`, the capacity of the Replay Buffer (for now, RBs are FIFO)
- `replay.warmup`, the number of initial samples to draw before starting to train

**Making the best of GPU**

As alluded to above, a main bottleneck in training GFlowNets is the _sampling_ of new trajectories. This requires doing "small" forward passes over and over to compute $P_F$, sampling actions, and taking environment steps. This also involves lots of CPU computation, which would normally keep the GPU unused. This library distributes this CPU work in workers in a fairly transparent way. Using a `GFNTrainer` sets this up for you! **All that needs to be done is to set `cfg.num_workers` to an appropriate value.**

Another (unfortunately) important choice is to decide how workers should communicate with each other. From fastest to slowest (but least to most convenient):
- Using shared buffers; requires setting `cfg.mp_buffer_size` to the number of bytes to use (~per worker) to create those fixed buffers.
- Using pickle; (un)surprisingly, torch is bad at transferring lots of small tensors, and so a valid strategy is to set `cfg.pickle_mp_messages = True`.
- Doing nothing; this relies on torch's internal serialization and memory sharing mechanisms, which are really not optimized for our use case but can handle anything.

Finally, another big factor in GPU usage is batch size. YMMV, but generally speaking the larger batch size that can be made to fit in GPU memory, the better.